# Raghav Gupta

March 2020

These are code excerpts from a course on optimization modelling

## Problem 1: Lockbox

A major credit card company (call it “MasterDebt”) receives checks from all different regions in the country on a daily basis.  Once these checks are mailed, the time a check spends in the mail (called “float”) creates loss for MasterDebt, for as soon as they receive the checks they can cash them and collect interest on the funds.  MasterDebt can make 15% annual interest on their cash holdings (since that’s what they charge their customers in credit card interest).

In order to reduce the amount of float loss for these checks, MasterDebt is considering opening “Lockbox” locations across the country where the checks can be received and processed.

The annual interest lost can be computed for each region-lockbox location by taking the average daily payments times the float time and multiplying by fifteen percent.  For example, if payments from the Central region are sent to New York, then on any given day there is an average of $135,000 of undeposited checks, which costs MasterDebt $20,250 annually in interest.

Where should MasterDebt open lockbox locations in order to save the most money each year?
Which regions should be assigned to those lockbox locations?  



In [ ]:
# Import Packages

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
from scipy.stats import norm
from pylab import *
from pyomo.environ import *

In [ ]:
# Read Tables

interest = pd.read_excel("lockbox.xlsx", sheet_name = "interesta", index_col=0)
annual = pd.read_excel("lockbox.xlsx", sheet_name = "costa", index_col=0)
matrix = pd.read_excel("lockbox.xlsx", sheet_name = "costm", index_col=0)
floats = pd.read_excel("lockbox.xlsx", sheet_name = "floats", index_col=0)
payment = pd.read_excel("lockbox.xlsx", sheet_name = "payment", index_col=0)

In [ ]:
# Construct the objective function

model = ConcreteModel()

C = interest.keys()
I = interest.index

model.x = Var(I,C, domain=Binary)
model.y = Var(C, domain=Binary)

model.cost = Objective(expr = sum([model.x[i,c]*interest.loc[i,c] for i in I for c in C]) + sum([model.y[c]*annual.loc['OpCost',c] for c in C]), sense=minimize)

In [ ]:
# Construct the constraints

model.cons = ConstraintList()

for i in I:
        model.cons.add(sum([model.x[i,c] for c in C]) ==1)
for c in C:
    for i in I:
        model.cons.add(model.x[i,c] <= model.y[c])

In [ ]:
# Run the model

SolverFactory('glpk').solve(model)
model.display()